In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages
import statsmodels.stats.multitest as smm
import numpy as np
#from bioinfokit import analys, visuz
from matplotlib.patches import Circle
from matplotlib.patheffects import withStroke
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
import matplotlib

color_palette = sns.color_palette("colorblind")

sns.set_theme(palette=color_palette, font="Arial", font_scale=1.0, style="white")

plt.style.use('../mpl_style_template/poster.mplstyle')

"""matplotlib.rcParams.update({'font.size': 8, 'axes.titlesize': 8, 'axes.labelsize': 8,
                           'xtick.labelsize': 8, 'ytick.labelsize': 8, 'legend.fontsize': 8,
                            'text.usetex': False, 'svg.fonttype': 'none',
                           'xtick.direction': 'out', 'ytick.direction': 'out',
                            'xtick.bottom': True, 'ytick.left': True,})"""

#matplotlib.rcParams.update({'axes.labelsize': 16,'xtick.labelsize': 16,})
import os


In [2]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 25)
#pd.set_option('display.max_colwidth', 300)
#pd.set_option('format.precision', 2)




In [3]:
#Load "targets_and_families from https://www.guidetopharmacology.org/download.jsp

targets_families=pd.read_csv("targets_and_families_220518.csv", header=1, \
                             usecols=['Type', \
                                      'Family id', \
                                      'Family name', \
                                      'Target id', \
                                      'Target name',\
                                      'Subunits', \
                                      'Target systematic name', \
                                      'Target abbreviated name', \
                                      'synonyms', \
                                      'MGI id',\
                                      'MGI symbol', \
                                      'MGI name',\
                                      'Mouse Ensembl Gene']) 

#drop rows where no Ensembl Gene is available
targets_families.dropna(subset=['Mouse Ensembl Gene'], inplace=True)
targets_families

Type  Family id                    Family name  Target id  \
0              gpcr          1  5-Hydroxytryptamine receptors          1   
1              gpcr          1  5-Hydroxytryptamine receptors          2   
2              gpcr          1  5-Hydroxytryptamine receptors          3   
4              gpcr          1  5-Hydroxytryptamine receptors          5   
5              gpcr          1  5-Hydroxytryptamine receptors          6   
...             ...        ...                            ...        ...   
3215  other_protein        858                       Tubulins       2641   
3217  other_protein        904     Tumour-associated antigens       2959   
3219  other_protein        904     Tumour-associated antigens       3009   
3220  other_protein        904     Tumour-associated antigens       2837   
3221  other_protein        903  WD repeat-containing proteins       2831   

                                       Target name Subunits  \
0                       5-HT<sub>1A</sub> receptor      NaN   
1                       5-HT<sub>1B</sub> receptor      NaN   
2                       5-HT<sub>1D</sub> receptor      NaN   
4                       5-HT<sub>1F</sub> receptor      NaN   
5                       5-HT<sub>2A</sub> receptor      NaN   
...                                            ...      ...   
3215                     tubulin beta 4B class IVb      NaN   
3217                                    glypican 3      NaN   
3219                      trophoblast glycoprotein      NaN   
3220  tumor associated calcium signal transducer 2      NaN   
3221                            WD repeat domain 5      NaN   

     Target systematic name Target abbreviated name  \
0                       NaN                     NaN   
1                       NaN                     NaN   
2                       NaN                     NaN   
4                       NaN                     NaN   
5                       NaN                     NaN   
...                     ...                     ...   
3215                    NaN                     NaN   
3217                    NaN                     NaN   
3219                    NaN                     NaN   
3220                    NaN                     NaN   
3221                    NaN                     NaN   

                                               synonyms       MGI id  \
0     ADRBRL1|5-HT1A|ADRB2RL1|serotonin receptor 1A|...    MGI:96273   
1     5-HT1B|5-HT1DB|HTR1D2|5-HT1B serotonin recepto...    MGI:96274   
2     5-HT<sub>1D&alpha;</sub>|HTRL|5-HT1D|HT1DA|ser...    MGI:96276   
4     5-HT<sub>1E&beta;</sub>|5-HT<sub>6</sub>|5-HT1...    MGI:99842   
5     'D' receptor|5Ht-2|serotonin 5HT-2 receptor|se...   MGI:109521   
...                                                 ...          ...   
3215  tubulin, beta 4B class IVb|tubulin|tubulin, be...  MGI:1915472   
3217                                                NaN   MGI:104903   
3219  5T4|5T4-Ag|5T4 oncofetal antigen|Wnt-activated...  MGI:1341264   
3220  M1S1|EGP-1|TROP2|TROP-2|GA733-1|RS7 antigen|tu...  MGI:1861606   
3221  CFAP89|cilia and flagella associated protein 8...  MGI:2155884   

     MGI symbol                                      MGI name  \
0         Htr1a   5-hydroxytryptamine (serotonin) receptor 1A   
1         Htr1b   5-hydroxytryptamine (serotonin) receptor 1B   
2         Htr1d   5-hydroxytryptamine (serotonin) receptor 1D   
4         Htr1f   5-hydroxytryptamine (serotonin) receptor 1F   
5         Htr2a   5-hydroxytryptamine (serotonin) receptor 2A   
...         ...                                           ...   
3215     Tubb4b                    tubulin, beta 4B class IVB   
3217       Gpc3                                    glypican 3   
3219       Tpbg                      trophoblast glycoprotein   
3220    Tacstd2  tumor-associated calcium signal transducer 2   
3221       Wdr5                            WD repeat domain 5   

      Mouse Ensembl Gene  
0     ENSMU

In [4]:
#unique family types
targets_families["Type"].unique()

array(['gpcr', 'nhr', 'vgic', 'lgic', 'other_ic', 'catalytic_receptor',
       'enzyme', 'transporter', 'other_protein'], dtype=object)

In [5]:
#number of unique genes 
#75 genes have more than one entry resulting in 79 duplicates
targets_families["Mouse Ensembl Gene"].nunique()

2830

In [9]:
#genes with more than one entry
ids = targets_families["Mouse Ensembl Gene"]
print(len(targets_families[ids.isin(ids[ids.duplicated()])].sort_values("Mouse Ensembl Gene")))
targets_families[ids.isin(ids[ids.duplicated()])].sort_values("Mouse Ensembl Gene")

75


Type  Family id  \
1410              enzyme        766   
1289              enzyme        249   
1106              enzyme        840   
1918              enzyme        920   
3196       other_protein       1000   
3147       other_protein        949   
2962       other_protein        852   
1688              enzyme        272   
1807              enzyme        737   
1609              enzyme        799   
962   catalytic_receptor        334   
822   catalytic_receptor        953   
2081              enzyme        270   
1515              enzyme        265   
2004              enzyme        782   
2344              enzyme        674   
1797              enzyme        749   
1686              enzyme        272   
1408              enzyme        766   
1288              enzyme        249   
1529              enzyme        256   
1411              enzyme        766   
3156       other_protein        949   
2996       other_protein        852   
1977              enzyme        673   
1992              enzyme        781   
1089              enzyme        877   
1924              enzyme        920   
1618              enzyme        799   
1973              enzyme        276   
1915              enzyme        920   
1098              enzyme        840   
1181              enzyme        908   
1921              enzyme        920   
1925              enzyme        920   
1090              enzyme        877   
3150       other_protein        949   
2977       other_protein        852   
1916              enzyme        920   
1157              enzyme        846   
1234              enzyme        847   
1600              enzyme        799   
1254              enzyme        765   
2057              enzyme        275   
1616              enzyme        799   
1922              enzyme        920   
1113              enzyme        840   
1438              enzyme        471   
515                 vgic         78   
1197              enzyme        850   
2199              enzyme        917   
2345              enzyme        674   
2005              enzyme        782   
1920              enzyme        920   
1142              enzyme        843   
2990       other_protein        852   
3154       other_protein        949   
2982       other_protein        852   
3151       other_protein        949   
2961       other_protein        852   
3146       other_protein        949   
3155       other_protein        949   
2991       other_protein        852   
1987              enzyme        781   
1980              enzyme        671   
2003              enzyme        782   
2343              enzyme        674   
1570              enzyme        764   
1526              enzyme        256   
2160              enzyme        751   
1310              enzyme        853   
516                 vgic         78   
1439              enzyme        471   
1990              enzyme        781   
1975              enzyme        673   
1601              enzyme        799   
1255              enzyme        765   
1233              enzyme        849   
1101              enzyme        840   
2000              enzyme        782   
2346              enzyme        675   
2925       other_protein       1021   
2965       other_protein        852   
1443              enzyme        476   
2119              enzyme        609   
1512              enzyme        265   
2080              enzyme        270   
2987       other_protein        852   
3152       other_protein        949   
1981              enzyme        671   
1988              enzyme        781   
672                 lgic         76   
564                 vgic         81   
1979              enzyme        671   
1986              enzyme        781   
2200              enzyme        917   
1198              enzyme        850   
2964       other_protein        852   
3204       other_protein       1047   
1923              enzyme        920   
1088              enzyme        877   
1314              enzyme        853   
216

In [10]:
# load results table from R DESeq analysis (gene-level, LFC shrinkage)
results_genes=pd.read_csv("shrink_Anno.csv")  
results_genes.rename(columns={"Unnamed: 0": "GENEID"}, inplace = True)

results_genes#.dropna(axis=0, subset=["SYMBOL"])
results_genes #all GENEIDs are unique

GENEID     baseMean  log2FoldChange     lfcSE  \
0      ENSMUSG00000000001  6199.665736        0.290552  0.058169   
1      ENSMUSG00000000003     0.000000             NaN       NaN   
2      ENSMUSG00000000028   135.410049        0.023191  0.069728   
3      ENSMUSG00000000037    26.062083       -0.009071  0.065583   
4      ENSMUSG00000000049     1.727977        0.003275  0.065297   
...                   ...          ...             ...       ...   
35677  ENSMUSG00000118670     2.452789       -0.002622  0.065113   
35678  ENSMUSG00000118671   254.637367       -0.003083  0.062567   
35679  ENSMUSG00000118672     0.000000             NaN       NaN   
35680  ENSMUSG00001074846     0.000000             NaN       NaN   
35681  ENSMUSG00002076083   599.965868       -0.027340  0.060217   

             pvalue      padj GENENAME  SYMBOL  ENTREZID  
0      4.072323e-08  0.000096    Gnai3   Gnai3   14679.0  
1               NaN       NaN     Pbsn    Pbsn   54192.0  
2      1.041882e-01  0.564404    Cdc45   Cdc45   12544.0  
3      2.880739e-01  0.786633    Scml2   Scml2  107815.0  
4      1.904263e-01       NaN     Apoh    Apoh   11818.0  
...             ...       ...      ...     ...       ...  
35677  5.924836e-01       NaN      NaN     NaN       NaN  
35678  8.624442e-01  0.982947    EPPK1   EPPK1       NaN  
35679           NaN       NaN     Muc4    Muc4  140474.0  
35680           NaN       NaN      NaN     NaN   68265.0  
35681  3.859461e-01  0.833101   Klhl17  Klhl17  231003.0  

[35682 rows x 9 columns]

In [11]:
# merge with targets_family information
data = results_genes.merge(targets_families, how = "left", 
                           left_on='GENEID', 
                           right_on='Mouse Ensembl Gene', 
                           validate="one_to_many")

In [12]:
ids = data["GENEID"]
data[ids.isin(ids[ids.duplicated()])]# genes that have mor then one entry and their dupicates

GENEID       baseMean  log2FoldChange     lfcSE    pvalue  \
35     ENSMUSG00000000214    3370.078812        0.019837  0.067303  0.233710   
36     ENSMUSG00000000214    3370.078812        0.019837  0.067303  0.233710   
595    ENSMUSG00000003500     573.917972        0.019989  0.064158  0.462087   
596    ENSMUSG00000003500     573.917972        0.019989  0.064158  0.462087   
727    ENSMUSG00000004609     101.455681       -0.012869  0.065528  0.349565   
728    ENSMUSG00000004609     101.455681       -0.012869  0.065528  0.349565   
729    ENSMUSG00000004609     101.455681       -0.012869  0.065528  0.349565   
1665   ENSMUSG00000015889     968.446988        0.015611  0.046331  0.636676   
1666   ENSMUSG00000015889     968.446988        0.015611  0.046331  0.636676   
1667   ENSMUSG00000015889     968.446988        0.015611  0.046331  0.636676   
1820   ENSMUSG00000017652      66.215676       -0.036876  0.079004  0.029393   
1821   ENSMUSG00000017652      66.215676       -0.036876  0.079004  0.029393   
1871   ENSMUSG00000017969    1090.734254       -0.003753  0.062269  0.841257   
1872   ENSMUSG00000017969    1090.734254       -0.003753  0.062269  0.841257   
1949   ENSMUSG00000018547    1023.926112        0.030644  0.051344  0.369794   
1950   ENSMUSG00000018547    1023.926112        0.030644  0.051344  0.369794   
2073   ENSMUSG00000019278    1780.719453       -0.005080  0.058441  0.839329   
2074   ENSMUSG00000019278    1780.719453       -0.005080  0.058441  0.839329   
2279   ENSMUSG00000020051       1.128053        0.000956  0.065173  0.729532   
2280   ENSMUSG00000020051       1.128053        0.000956  0.065173  0.729532   
2367   ENSMUSG00000020182     526.615149        0.017300  0.066506  0.608818   
2368   ENSMUSG00000020182     526.615149        0.017300  0.066506  0.608818   
2494   ENSMUSG00000020399     125.149222        0.014127  0.060394  0.588281   
2495   ENSMUSG00000020399     125.149222        0.014127  0.060394  0.588281   
2597   ENSMUSG00000020573     109.800233        0.010029  0.062727  0.618716   
2598   ENSMUSG00000020573     109.800233        0.010029  0.062727  0.618716   
2646   ENSMUSG00000020649     146.414058       -0.010712  0.065448  0.353597   
2647   ENSMUSG00000020649     146.414058       -0.010712  0.065448  0.353597   
2760   ENSMUSG00000020828     289.586592       -0.073551  0.106901  0.049109   
2761   ENSMUSG00000020828     289.586592       -0.073551  0.106901  0.049109   
3247   ENSMUSG00000021707     281.565363       -0.021202  0.065717  0.328595   
3248   ENSMUSG00000021707     281.565363       -0.021202  0.065717  0.328595   
3337   ENSMUSG00000021871     236.296181        0.006207  0.062239  0.753384   
3338   ENSMUSG00000021871     236.296181        0.006207  0.062239  0.753384   
3583   ENSMUSG00000022292     855.532478       -0.034281  0.068397  0.228276   
3584   ENSMUSG00000022292     855.532478       -0.034281  0.068397  0.228276   
3960   ENSMUSG00000022901      93.884665       -0.007141  0.063637  0.655179   
3961   ENSMUSG00000022901      93.884665       -0.007141  0.063637  0.655179   
3985   ENSMUSG00000022962    1594.549654        0.002456  0.047690  0.940751   
3986   ENSMUSG00000022962    1594.549654        0.002456  0.047690  0.940751   
3987   ENSMUSG00000022962    1594.549654        0.002456  0.047690  0.940751   
4113   ENSMUSG00000023328      79.587815        0.002211  0.064176  0.852475   
4114   ENSMUSG00000023328      79.587815        0.002211  0.064176  0.852475   
4180   ENSMUSG00000023913     402.226239       -0.030497  0.064553  0.301006   
4181   ENSMUSG00000023913     402.226239       -0.030497  0.064553  0.301006   
4272   ENSMUSG00000024066    6190.950752        0.007652  0.065187  0.430037   
4273   ENSMUSG00000024066    6190.950752        0.007652  0.065187  0.430037   
4633   ENSMUSG00000024727       2.515365        0.000949  0.065128  0.797677   
4634   ENSMUSG00000024727       2.515365        0.000949  0.065128  0.797677   
4659   ENSMUSG00000

In [13]:
#formatting
data["padj_rounded"] = data["padj"].map('{:.1e}'.format)
data["GENENAME_padj"] = data["GENENAME"] + "\np=" + data["padj_rounded"].astype(str) +""
data["-log10_padj"] = -np.log10(data["padj"])

In [15]:
matplotlib.rcParams.update({'font.size': 8.2, 'axes.titlesize': 8.2, 'axes.labelsize': 8.2,
                           'xtick.labelsize': 8.2, 'ytick.labelsize': 8.2, 'legend.fontsize': 8.2,
                            'text.usetex': False, 'svg.fonttype': 'none',
                           'xtick.direction': 'out', 'ytick.direction': 'out',
                            'xtick.bottom': True, 'ytick.left': True, 'axes.linewidth':1})


matplotlib.rcParams.update({'ytick.left': True})
padj_thr = 0.05
lfc_thr = 1

type_list = ["vgic", "lgic", "other_ic", "transporter", "gpcr"]
anno_list = ["Star", "Agtr1a"]
anno_list_3 = [ "Cyp11b2", "Agtr1b", "Agtr2"]
anno_list_2 = ["Cacna1c", "Cacna1d", "Cacna1g"]



#fig, ax = plt.subplots(figsize=(3.4,4))
fig, ax = plt.subplots(figsize=(3.4,7.9))



data_all = data.dropna(subset=["log2FoldChange", "padj", "GENENAME"])
data_all_up = data_all[(data_all["log2FoldChange"] > lfc_thr) & (data_all["padj"] < 0.05)]
data_all_down = data_all[(data_all["log2FoldChange"] < -lfc_thr) & (data_all["padj"] < 0.05)]

data_all_up_in_list = data_all_up[data_all_up["Type"].isin(type_list)]
data_all_down_in_list = data_all_down[data_all_down["Type"].isin(type_list)]



ax.scatter(x=data_all["log2FoldChange"],\
            y=data_all["-log10_padj"], color ="silver", s=22, label='not significant')

ax.scatter(x=data_all_up["log2FoldChange"],\
            y=data_all_up["-log10_padj"], color="firebrick", s=22, label='significantly upregulated' )

ax.scatter(x=data_all_down["log2FoldChange"],\
            y=data_all_down["-log10_padj"], color="mediumblue", s=22, label='significantly downregulated' )




ax.set_ylabel("-log10_padj")
ax.set_xlabel("log2FoldChange")





for i in data_all_down["GENENAME"].values:
    gene = data_all_down[data_all_down["GENENAME"] == i]
    y = gene["-log10_padj"].values[0]
    x = gene["log2FoldChange"].values[0]
    
    ax.annotate(gene["GENENAME"].values[0], xy=(x, y), xytext=(x, y), fontsize = 8, 
                    style = "italic", ha="right")

    
for i in data_all_up["GENENAME"].values:
    gene = data_all_up[data_all_up["GENENAME"] == i]
    y = gene["-log10_padj"].values[0]
    x = gene["log2FoldChange"].values[0]
        
    ax.annotate(gene["GENENAME"].values[0], xy=(x, y), xytext=(x, y), fontsize = 8, 
                    style = "italic",)
    

  
    
    

### plot data outside range    ####################    
plot_limits=2.8

data_outside_lim_up = data_all[data_all["log2FoldChange"] >= plot_limits]      
data_outside_lim_down = data_all[data_all["log2FoldChange"] <= -plot_limits]      
        
#ax.scatter(x=[2.96,2.96],\
#            y=data_outside_lim_up["-log10_padj"], color ="grey", s=10, label='LFC > 3', marker=">", facecolor='none')    
    
#ax.scatter(x=[-2.96],\
#            y=data_outside_lim_down["-log10_padj"], color ="grey", s=10, label='LFC < -3', marker="<", facecolor='none')   

###################################################

ax.set_ylim(-0.05, 7.5)
ax.set_xlim(-plot_limits, plot_limits)

ax.set_ylabel("-log${_{10}}$(p-value)")   
ax.set_xlabel("log${_{2}}$(Fold change)")       

#ax.legend()

ax.axhline(-np.log10(padj_thr),0,10, ls="--", color="grey", lw=1)
ax.axvline(-1,0,10, ls="--", color="grey",lw=1)
ax.axvline(1,0,10, ls="--", color="grey",lw=1)

fig.savefig("RNASeq_volcano_shrink.png", dpi=1200)
fig.savefig("RNASeq_volcano_shrink.svg", dpi=1200)

In [18]:
data_DE_all_sortL2FC = pd.concat([data_all_up.sort_values(by=["log2FoldChange"], ascending=False),
           data_all_down.sort_values(by=["log2FoldChange"], ascending=False)])

data_DE_all_sortL2FC

GENEID     baseMean  log2FoldChange     lfcSE  \
9952   ENSMUSG00000035202  4680.061440        1.820094  0.697889   
3105   ENSMUSG00000021453   242.977276        1.492890  0.228217   
18880  ENSMUSG00000072845    23.235980        1.455469  0.478889   
15266  ENSMUSG00000053007  4068.907721        1.068352  0.314270   
16124  ENSMUSG00000057440   837.041742        1.061426  0.238488   
5334   ENSMUSG00000026064   241.629377        1.055277  0.236007   
12417  ENSMUSG00000042041   148.059701        1.044430  0.247909   
9139   ENSMUSG00000032803  7885.570112        1.035186  0.404309   
4030   ENSMUSG00000023033    48.204375       -1.032317  0.367088   
27433  ENSMUSG00000099034    36.526190       -1.058356  0.225508   
8712   ENSMUSG00000032010   527.493498       -1.063745  0.307350   
4303   ENSMUSG00000024112  2327.226910       -1.348250  0.262770   

             pvalue          padj       GENENAME         SYMBOL  ENTREZID  \
9952   3.046884e-04  3.647998e-02          Lars2          Lars2  102436.0   
3105   3.024359e-12  4.272815e-08        Gadd45g        Gadd45g   23882.0   
18880  9.061068e-05  2.209341e-02      Tmprss11a      Tmprss11a  194597.0   
15266  2.501755e-05  8.368726e-03          Creb5          Creb5  231991.0   
16124  3.823155e-07  5.401354e-04           Mpp7           Mpp7   75739.0   
5334   3.465161e-07  5.401354e-04         Ptp4a1         Ptp4a1   19243.0   
12417  1.067232e-06  9.423659e-04  2010003K11Rik  2010003K11Rik   69861.0   
9139   3.896123e-04  4.234187e-02           Cdv3           Cdv3  321022.0   
4030   1.401763e-04  2.571963e-02          Scn8a          Scn8a   20273.0   
27433  1.218341e-07  2.361395e-04  2810039B14Rik  2810039B14Rik       NaN   
8712   1.787938e-05  7.654543e-03           Usp2           Usp2   53376.0   
4303   1.090519e-08  3.851712e-05        Cacna1h        Cacna1h   58226.0   

         Type  Family id                                       Family name  \
9952      NaN        NaN                                               NaN   
3105      NaN        NaN                                               NaN   
18880     NaN        NaN                                               NaN   
15266     NaN        NaN                                               NaN   
16124     NaN        NaN                                               NaN   
5334      NaN        NaN                                               NaN   
12417     NaN        NaN                                               NaN   
9139      NaN        NaN                                               NaN   
4030     vgic       82.0    Voltage-gated sodium channels (Na<sub>V</sub>)   
27433     NaN        NaN                                               NaN   
8712   enzyme      730.0                  C19: Ubiquitin-specific protease   
4303     vgic       80.0  Voltage-gated calcium channels  (Ca<sub>V</sub>)   

       Target id                     Target name Subunits  \
9952         NaN                             NaN      NaN   
3105         NaN                             NaN      NaN   
18880        NaN                             NaN      NaN   
15266        NaN                             NaN      NaN   
16124        NaN                             NaN      NaN   
5334         NaN                             NaN      NaN   
12417        NaN                             NaN      NaN   
9139         NaN                             NaN      NaN   
4030       583.0               Na<sub>v</sub>1.6      NaN   
27433        NaN                             NaN      NaN   
8712      2430.0  ubiquitin specific peptidase 2      NaN   
4303       536.0               Ca<sub>v</sub>3.2      NaN   

      Target systematic name Target abbreviated name  \
9952                     NaN                     NaN   
3105                     NaN                     NaN   
18880                    NaN                     NaN   
15266                    NaN                     NaN   
16124                    NaN               

In [19]:
data_DE_all_sortL2FC.sort_values(by=["log2FoldChange"], ascending=True)[["GENENAME", 
                                                                         "log2FoldChange", 
                                                                         "padj_rounded"]]\
.round(3)\
.to_excel("data_l2fc_p_DEG_230413.xlsx")

In [20]:
genes_of_interest = ["Cacna1h", 
                     "Cacna1c",
                     "Cacna1d",
                     "Star", 
                     "Agtr1a", 
                     "Agtr1b", 
                     "Agtr2", 
                     "Cyp11b2",
                     "Kcnk3",#TASK channels
                     "Kcnk5",
                     "Kcnk9",
                     "Kcnk15",
                     "Kcnma1",#BK channel
                     "Kcnn1", #SK channels
                     "Kcnn2",
                     "Kcnn3",
                     "Kcnn4",
                     "Kcnj3", #GIRK channels
                     "Kcnj6",
                     "Kcnj9"
                    ]

In [21]:
data_all_goi = data_all[data_all["GENENAME"].isin(genes_of_interest)]
data_all_goi.sort_values(by=["log2FoldChange"], ascending=True)

GENEID       baseMean  log2FoldChange     lfcSE  \
4303   ENSMUSG00000024112    2327.226910       -1.348250  0.262770   
15647  ENSMUSG00000054988     405.449547       -0.021236  0.061451   
499    ENSMUSG00000002908      73.401200       -0.014620  0.066091   
4096   ENSMUSG00000023243     273.950895       -0.013202  0.066831   
3776   ENSMUSG00000022589    6756.439472       -0.009134  0.063916   
10444  ENSMUSG00000036760    6020.801974       -0.006955  0.065211   
18026  ENSMUSG00000068122     543.322560       -0.006643  0.064549   
17342  ENSMUSG00000063142    2307.017567       -0.005564  0.062379   
15521  ENSMUSG00000054342      19.764408        0.000481  0.064213   
137    ENSMUSG00000000794      83.814982        0.003009  0.063510   
1680   ENSMUSG00000015968     358.570892        0.003163  0.063857   
14922  ENSMUSG00000051331    3778.994018        0.037301  0.063145   
15551  ENSMUSG00000054477    2264.677672        0.042398  0.072939   
14383  ENSMUSG00000049265   24269.521923        0.068350  0.078512   
8440   ENSMUSG00000031574  382182.964278        0.347405  0.090991   
14349  ENSMUSG00000049115    5805.410148        0.431220  0.175557   

             pvalue      padj GENENAME   SYMBOL  ENTREZID    Type  Family id  \
4303   1.090519e-08  0.000039  Cacna1h  Cacna1h   58226.0    vgic       80.0   
15647  4.456527e-01  0.866050   Agtr1b   Agtr1b   11608.0     NaN        NaN   
499    2.842965e-01  0.782611    Kcnn1    Kcnn1   84036.0    vgic       69.0   
4096   6.920826e-02  0.483493    Kcnk5    Kcnk5   16529.0    vgic       79.0   
3776   5.393133e-01  0.904201  Cyp11b2  Cyp11b2   13072.0  enzyme      266.0   
10444  4.576163e-01  0.869831    Kcnk9    Kcnk9  223604.0    vgic       79.0   
18026  5.777480e-01  0.916383    Agtr2    Agtr2   11609.0    gpcr        6.0   
17342  7.697755e-01  0.967658   Kcnma1   Kcnma1   16531.0    vgic       69.0   
15521  9.672142e-01  0.997111    Kcnn4    Kcnn4   16534.0    vgic       69.0   
137    8.405325e-01  0.980841    Kcnn3    Kcnn3  140493.0    vgic       69.0   
1680   8.175124e-01  0.975517  Cacna1d  Cacna1d   12289.0    vgic       80.0   
14922  2.683993e-01  0.770839  Cacna1c  Cacna1c   12288.0    vgic       80.0   
15551  1.679960e-01  0.666886    Kcnn2    Kcnn2  140492.0    vgic       69.0   
14383  7.919355e-02  0.509029    Kcnk3    Kcnk3   16527.0    vgic       79.0   
8440   1.348753e-05  0.006805     Star     Star   20845.0     NaN        NaN   
14349  5.081941e-04  0.047548   Agtr1a   Agtr1a   11607.0     NaN        NaN   

                                             Family name  Target id  \
4303    Voltage-gated calcium channels  (Ca<sub>V</sub>)      536.0   
15647                                                NaN        NaN   
499    Calcium- and sodium-activated potassium channe...      381.0   
4096   Two-pore domain potassium channels  (K<sub>2P<...      517.0   
3776       CYP11, CYP17, CYP19, CYP20 and CYP21 families     1360.0   
10444  Two-pore domain potassium channels  (K<sub>2P<...      520.0   
18026                              Angiotensin receptors       35.0   
17342  Calcium- and sodium-activated potassium channe...      380.0   
15521  Calcium- and sodium-activated potassium channe...      384.0   
137    Calcium- and sodium-activated potassium channe...      383.0   
1680    Voltage-gated calcium channels  (Ca<sub>V</sub>)      530.0   
14922   Voltage-gated calcium channels  (Ca<sub>V</sub>)      529.0   
15551  Calcium- and sodium-activated potassium channe...      382.0   
14383  Two-pore domain potassium channels  (K<sub>2P<...      515.0   
8440                                                 NaN        NaN   
14349                                                NaN        NaN   

                   Target name Subunits Target systematic name  \
4303         Ca<sub>v</sub>3.2      NaN                    NaN   
15647                      NaN      NaN                    NaN   
499          K<sub>Ca</sub>2.1      NaN                    Na

In [23]:
data_all_goi[["GENENAME", "log2FoldChange", "padj_rounded"]].round(3).to_excel("data_l2fc_p_goi_230413.xlsx")